In [10]:
import os
os.chdir("/home/b.weinstein/DeepTreeAttention")
from src.data import read_config, TreeData
from src.models import multi_stage
from pytorch_lightning import Trainer
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import cProfile

In [11]:
config = read_config("config.yml")
config["crop_dir"] = os.path.join(config["data_dir"], config["use_data_commit"])
config["train_test_commit"] = "83765201df47ba137ae73dc3d531c93c68367dea"
config["preload_images"] = True
config["preload_image_dict"] = True
config

{'rgb_sensor_pool': '/orange/ewhite/NeonData/*/DP3.30010.001/**/Camera/**/*.tif',
 'HSI_sensor_pool': '/orange/ewhite/NeonData/*/DP3.30006.001/**/Reflectance/*.h5',
 'CHM_pool': '/orange/ewhite/NeonData/**/CanopyHeightModelGtif/*.tif',
 'HSI_tif_dir': '/orange/ewhite/b.weinstein/DeepTreeAttention/Hyperspectral_tifs/year/',
 'min_stem_diameter': 10,
 'min_train_samples': 48,
 'min_test_samples': 24,
 'max_samples_per_class': 1000,
 'iterations': 200,
 'min_CHM_height': 1,
 'max_CHM_diff': 4,
 'CHM_height_limit': 8,
 'megaplot_dir': '/orange/idtrees-collab/megaplot/',
 'use_data_commit': '04a3c3001163490690ad7c90d7317ddf',
 'data_dir': '/blue/ewhite/b.weinstein/DeepTreeAttention/',
 'convert_h5': True,
 'replace_bounding_boxes': False,
 'replace_crops': False,
 'train_test_commit': '83765201df47ba137ae73dc3d531c93c68367dea',
 'existing_test_csv': None,
 'image_size': 30,
 'preload_images': True,
 'preload_image_dict': True,
 'workers': 0,
 'taxonomic_csv': '/home/b.weinstein/DeepTreeAtte

In [12]:
data_module = TreeData(
    csv_file="/home/b.weinstein/DeepTreeAttention/data/raw/neon_vst_data_2022.csv",
    data_dir=config['crop_dir'],
    experiment_id="profiler",
    config=config,
    client=None,
    site="['OSBS', 'JERC', 'TALL', 'DSNY']",
    comet_logger=None)

Loading previous data commit 04a3c3001163490690ad7c90d7317ddf
Loading a train-test split from /blue/ewhite/b.weinstein/DeepTreeAttention/04a3c3001163490690ad7c90d7317ddf/83765201df47ba137ae73dc3d531c93c68367dea_['OSBS', 'JERC', 'TALL', 'DSNY']
There are 14209 records for 17 species for 4 sites in filtered train
There are 1653 records for 17 species for 4 sites in test


In [13]:
m = multi_stage.MultiStage(
        train_df=data_module.train, 
        test_df=data_module.test, 
        config=data_module.config) 

In [14]:
trainer = Trainer(max_epochs=2, profiler="simple")
m.setup("fit")
m.current_level = "dominant_class"

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
#%prun individual, image, labels = next(iter(m.train_datasets["dominant_class"]))

In [16]:
#dl = m.train_dataloader()
#%prun batch = df = next(iter(dl))

In [17]:
trainer.fit(m)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]

  | Name          | Type       | Params
---------------------------------------------
0 | models        | ModuleDict | 7.5 M 
1 | level_metrics | ModuleDict | 0     
---------------------------------------------
7.5 M     Trainable params
0         Non-trainable params
7.5 M     Total params
30.139    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                   